In [1]:
%env HOME /home/ubuntu/

env: HOME=/home/ubuntu/


In [2]:
!pip3 install  gensim  
!pip3 install  tensorflow
!pip3 install  keras

  Using cached https://files.pythonhosted.org/packages/40/3d/89b27573f56abcd1b8c9598b240f53c45a3c79aa0924a24588e99716043b/gensim-3.8.0-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/19/b9/bda9781f0a74b90ebd2e046fde1196182900bd4a8e1ea503d3ffebc50e7c/numpy-1.17.0-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/73/fb/00a976f728d0d1fecfe898238ce23f502a721c0ac0ecfedb80e0d88c64e9/six-1.12.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/29/50/a552a5aff252ae915f522e44642bb49a7b7b31677f9580cfd11bcc869976/scipy-1.3.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/51/bd/23c926cd341ea6b7dd0b2a00aba99ae0f828be89d72b2190f27c11d4b7fb/requests-2.22.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ff/3e/2262936ad70fd6e7b8827d79d6508ce33e2ffb49bfca6fedc5fe4abd6f1c/boto3-1.9.215-py2.py3-none-any.whl
  Using cached https://file

  Using cached https://files.pythonhosted.org/packages/b2/86/095d2f7829badc207c893dd4ac767e871f6cd547145df797ea26baea4e2e/setuptools-41.2.0-py2.py3-none-any.whl
    100% |████████████████████████████████| 92kB 9.8MB/s eta 0:00:01
    100% |████████████████████████████████| 337kB 4.0MB/s eta 0:00:01
  Running setup.py bdist_wheel for termcolor ... done
  Stored in directory: /home/ubuntu/.cache/pip/wheels/7c/06/54/bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6
  Running setup.py bdist_wheel for absl-py ... done
  Stored in directory: /home/ubuntu/.cache/pip/wheels/ee/98/38/46cbcc5a93cfea5492d19c38562691ddb23b940176c14f7b48
  Running setup.py bdist_wheel for gast ... done
  Stored in directory: /home/ubuntu/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
  Running setup.py bdist_wheel for wrapt ... done
  Stored in directory: /home/ubuntu/.cache/pip/wheels/d7/de/2e/efa132238792efb6459a96e85916ef8597fcb3d2ae51590dfd
Successfully built termcolor absl-py ga

In [4]:
import numpy as np
np.random.seed(13)

import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda
from keras.utils.data_utils import get_file
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

import gensim

In [8]:
path = get_file('alice.txt', origin='http://www.gutenberg.org/files/11/11-0.txt')
#path = "/Users/cmauceri/PRG/notebooks/data/alice.txt"
corpus = open(path).readlines()[:300]
corpus = [sentence for sentence in corpus if sentence.count(' ') >= 2]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
corpus = tokenizer.texts_to_sequences(corpus)
nb_samples = sum(len(s) for s in corpus)
V = len(tokenizer.word_index) + 1
dim = 100
window_size = 2

180224/173595 [===============================] - 0s 0us/step


In [9]:
def generate_data(corpus, window_size, V):
    maxlen = window_size*2
    for words in corpus:
        L = len(words)
        for index, word in enumerate(words):
            contexts = []
            labels   = []            
            s = index - window_size
            e = index + window_size + 1
            
            contexts.append([words[i] for i in range(s, e) if 0 <= i < L and i != index])
            labels.append(word)

            x = sequence.pad_sequences(contexts, maxlen=maxlen)
            y = np_utils.to_categorical(labels, V)
            yield (x, y)

In [10]:
cbow = Sequential()
cbow.add(Embedding(input_dim=V, output_dim=dim, input_length=window_size*2))
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(dim,)))
cbow.add(Dense(V, activation='softmax'))

W0826 16:35:21.884895 140460345890624 deprecation_wrapper.py:119] From /home/ubuntu/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0826 16:35:21.887983 140460345890624 deprecation_wrapper.py:119] From /home/ubuntu/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [11]:
cbow.compile(loss='categorical_crossentropy', optimizer='adadelta')

W0826 16:35:25.575111 140460345890624 deprecation_wrapper.py:119] From /home/ubuntu/.local/lib/python3.6/site-packages/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0826 16:35:25.592408 140460345890624 deprecation_wrapper.py:119] From /home/ubuntu/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3576: The name tf.log is deprecated. Please use tf.math.log instead.



In [12]:
for ite in range(10):
    loss = 0.
    for x, y in generate_data(corpus, window_size, V):
        loss += cbow.train_on_batch(x, y)

    print(ite, loss)

W0826 16:35:30.216073 140460345890624 deprecation.py:323] From /home/ubuntu/.local/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0826 16:35:30.318779 140460345890624 deprecation_wrapper.py:119] From /home/ubuntu/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:1033: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



0 17412.570201396942
1 16152.255044937134
2 16024.153393626213
3 15933.200787901878
4 15825.323386132717
5 15722.679118514061
6 15632.638167619705
7 15551.51374682784
8 15473.663281828165
9 15396.448969751596


In [13]:
f = open('vectors.txt' ,'w')
f.write('{} {}\n'.format(V-1, dim))

8

In [14]:
vectors = cbow.get_weights()[0]
for word, i in tokenizer.word_index.items():
    str_vec = ' '.join(map(str, list(vectors[i, :])))
    f.write('{} {}\n'.format(word, str_vec))
f.close()

In [18]:
#!yes | pip3 uninstall ibm-cos-sdk-core 
#!pip3 install 'urllib3<1.25,>=1.20' --force-reinstall --upgrade  --no-cache-dir
#!python3 -m pip install --user boto3 --upgrade  --no-cache-dir
#!pip3 install awscli --force-reinstall --upgrade  --no-cache-dir
#!print("*")*30
#!pip3 install ibm-cos-sdk-core --upgrade   --no-cache-dir
print("*"* 100) 


****************************************************************************************************


In [15]:
import boto3
import gensim
w2v = gensim.models.KeyedVectors.load_word2vec_format('./vectors.txt', binary=False)

In [16]:
w2v.most_similar(positive=['rabbit'])

[('under', 0.7948803901672363),
 ('through', 0.7906328439712524),
 ('into', 0.7778702974319458),
 ('hole', 0.74982750415802),
 ('passage', 0.748092532157898),
 ('same', 0.7418962717056274),
 ('words', 0.7247999906539917),
 ('right', 0.7152602672576904),
 ('little', 0.7067757844924927),
 ('of', 0.7059166431427002)]

In [17]:
w2v.most_similar(positive=['alice'])

[('she', 0.6102246046066284),
 ('you', 0.6003879904747009),
 ('poor', 0.568252444267273),
 ('them', 0.5612145662307739),
 ('that', 0.5536032915115356),
 ('eat', 0.5530520677566528),
 ('now', 0.5518885850906372),
 ('dark', 0.5485525131225586),
 ('marked', 0.5482351779937744),
 ('thought', 0.5456332564353943)]